In [1]:
import pymongo
from sklearn.preprocessing import StandardScaler
import datetime 
import csv
import re
from collections import defaultdict
class Database(object):
    def __init__(self):
        self.connection = pymongo.MongoClient("localhost", 27017)
        self.db = self.connection["DSS4_SmallP"]
        self.mov = self.db["Movie_mojo"]
        self.act = self.db["actor"]
        self.dir = self.db['director']
        self.fra = self.db['franchise']
        
    def __getitem__(self, key):
        return self.db[key]
        
db = Database()

In [2]:
#imdb data load
with open('movie_metadata.csv', 'r') as fin:
    cin = csv.reader(fin)
    movie_imdb = [row for row in cin]
for n, i in enumerate(movie_imdb[0]):
    print(n,i)

0 color
1 director_name
2 num_critic_for_reviews
3 duration
4 director_facebook_likes
5 actor_3_facebook_likes
6 actor_2_name
7 actor_1_facebook_likes
8 gross
9 genres
10 actor_1_name
11 movie_title
12 num_voted_users
13 cast_total_facebook_likes
14 actor_3_name
15 facenumber_in_poster
16 plot_keywords
17 movie_imdb_link
18 num_user_for_reviews
19 language
20 country
21 content_rating
22 budget
23 title_year
24 actor_2_facebook_likes
25 imdb_score
26 aspect_ratio
27 movie_facebook_likes


In [28]:
dict_act = {}
actors = db.act.find()
for act in actors:
    dict_act[act['name']] = act['movie']
dict_dir = {}
directors = db.dir.find()
for dire in directors:
    dict_dir[dire['name']] = dire['movie']
dict_fra = {}
franchise = db.fra.find()
for fra in franchise:
    dict_fra[fra['name']] = fra['movie']

In [127]:
arr_imdb = []
dict_imdb = {}
for row in movie_imdb[1:]:
    row[11] = re.sub('\xc2\xa0','',row[11])
    row[11] = re.sub('[^\w]','',row[11])
    row[11] = row[11].lower()
    dict_imdb[row[11]] = {
        'actor' : [re.sub('[^\w]','',row[10]).lower(), 
                   re.sub('[^\w]','',row[6]).lower(), 
                   re.sub('[^\w]','',row[14]).lower()] ,
        'director' : [re.sub('[^\w]','',row[1]).lower()] ,
        'budget' : row[22] ,
        'year' : row[23]
        }
    arr_imdb.append(row[11])

# 영화 리스트 정리.

In [74]:
listsieze = 932

In [163]:
que = db.mov.find()

In [164]:
arr_mov = [x for x in db.mov.find()]
dict_mov = {}
for mov in arr_mov:
    name = re.sub('[^\w]','',mov['Mname'])
    name = name.lower()
    del(mov['_id'])
    dict_mov[name] = mov

In [121]:
dict_imdb

{'vforvendetta': {'actor': ['Natalie Portman',
   'Eddie Marsan',
   'Rupert Graves'],
  'budget': '54000000',
  'director': 'James McTeigue',
  'year': '2005'},
 'blackknight': {'actor': ['Tom Wilkinson',
   'Marsha Thomason',
   'Vincent Regan'],
  'budget': '50000000',
  'director': 'Gil Junger',
  'year': '2001'},
 'thefourthkind': {'actor': ['Milla Jovovich',
   'Will Patton',
   'Hakeem KaeKazim'],
  'budget': '10000000',
  'director': 'Olatunde Osunsanmi',
  'year': '2009'},
 'yentl': {'actor': ['Miriam Margolyes', 'Amy Irving', 'Steven Hill'],
  'budget': '12000000',
  'director': 'Barbra Streisand',
  'year': '1983'},
 'twinfallsidaho': {'actor': ['Sasha Alexander', 'William Katt', 'Jon Gries'],
  'budget': '500000',
  'director': 'Michael Polish',
  'year': '1999'},
 'joyeuxnoel': {'actor': ['Gary Lewis', 'Dany Boon', 'Ian Richardson'],
  'budget': '22000000',
  'director': 'Christian Carion',
  'year': '2005'},
 'igotthehookup': {'actor': ['Joe Estevez', 'Master P', 'Anthony

## 영화 배우,감독,프랜차이즈,제작비

In [171]:
def del_list(obj):
    arr = set([u'genres', u'people', u'franchises', u'', u'showdowns', u'indices', u'narrator', u'voice'])
    obj = set(obj)
    return list(obj.difference(arr))

arr_movies = []
for k, v in dict_mov.items():
    if len(v) > 4 :
        # 배우,감독,프랜차이즈
        Mname = v['Mname']
        Mname = re.sub('[^\w]','',Mname)
        Mname = Mname.lower()
        tmp = {
            'franchise' : [] ,
            'actor' : [] ,
            'director' : []
        }
        for tmpk, tmpv in tmp.items():
            for i in v[tmpk] :
                tmpv.append(re.sub('[^\w]','',i).lower())
            tmp[tmpk] = del_list(tmpv)
        for tmpk, tmpv in tmp.items():
            if len(tmpv) == 0 and tmpk != 'franchose':
                try :
                    tmp[tmpk] = dict_imdb[Mname][tmpk]
                except : 
                    pass
        v.update(tmp)
        v['Mname'] = Mname
        
        # 제작비
        if v['project_budget'] == 'NA' :
            try :
                v['project_budget'] = dict_imdb[Mname]['budget']
            except : 
                pass
        if v['project_budget'] == 'NA' and not(v['budget'] == 'NA' or v['budget'] == 'N/A') :
            print('error')
            break
        new_dict = {
            'Mname' : v['Mname'],
            'actor' : v['actor'],
            'project_budget' : v['project_budget'],
            'director' : v['director'],
            'franchise' : v['franchise'],
            'genre' : v['genre'],
            'grade' : v['grade'],
            'gross_kor' : v['gross_kor'],
            'nation' : v['nation'],
            'release_kor' : v['release_kor'],
            'runtime' : v['runtime']
        }
        arr_movies.append(new_dict)

In [174]:
db['Movie_after'].insert_many(arr_movies)

# 배우, 감독, 프랜차이즈 리스트 정리.

In [48]:
from collections import defaultdict
def get_data(dict_in):
    arr_new = []
    for k,v in dict_in.items():
        dict_new = {}
        k_new = re.sub('[^\w]','',k)
        k_new = k_new.lower()
        dict_new['name'] = k_new
        dict_new['movie'] = []
        for key_movie, value_movie in v.items():
            key_movie = re.sub('[^\w]','',key_movie)
            key_movie = key_movie.lower()
            if len(value_movie) == 4 and value_movie[3] != 'NA':
                value = {
                    'Mname' : key_movie,
                    'gross_kor' : value_movie[3],
                    'release_kor' : value_movie[2]
                }
                dict_new['movie'].append(value)
        arr_new.append(dict_new)
    return arr_new

In [49]:
def get_obj(objin, dict_imdb):
    count = 0
    count1 = 0
    objout = []
    for i in objin:
        dict_new = {}
        k = i['name']
        v = i['movie']
        k_new = re.sub('[^\w]','',k)
        k_new = k_new.lower()
        dict_new['name'] = k_new
        dict_new['movie'] = []
        if len(v) == 0:
            try :
                for mov in dict_imdb[k_new] :
                    try :
                        dict_mov[mov]['Mname'] = mov
                        if dict_mov[mov]['gross_kor'] != 'NA':
                            dict_new['movie'].append(dict_mov[mov])
                    except :
                        pass
    #            print(dict_imdb[k_new])
            except:
                pass
            objout.append(dict_new)
        else :
            objout.append(i)
    return objout
#objout

In [50]:
dict_imdb_actor = {}
for k, v in dict_imdb.items():
    k_new = re.sub('[^\w]','',k)
    k_new = k_new.lower()
    for actor in v['actor']:
        if actor:
            actor = re.sub('[^\w]','', actor)
            actor = actor.lower()
            try :
                if k_new not in dict_imdb_actor[actor]:
                    dict_imdb_actor[actor][k_new]={'year':v['year']}
            except:
                dict_imdb_actor[actor] = {k_new : {'year':v['year']}}
                
dict_imdb_dir = {}
for k, v in dict_imdb.items():
    k_new = re.sub('[^\w]','',k)
    k_new = k_new.lower()
    director = v['director']
    if director:
        director = re.sub('[^\w]','', director)
        director = director.lower()
        try :
            if k_new not in dict_imdb_dir[director]:
                dict_imdb_dir[director][k_new]={'year':v['year']}
        except:
            dict_imdb_dir[director] = {k_new : {'year':v['year']}}

In [52]:
arr = get_data(dict_act)
arr_act = get_obj(arr, dict_imdb_actor)
db["actor_after"].insert_many(arr_act)

In [53]:
arr = get_data(dict_dir)
arr_dir = get_obj(arr, dict_imdb_dir)
db["dirtor_after"].insert_many(arr_dir)

In [54]:
arr = get_data(dict_fra)
arr_fra = get_obj(arr, {})
db["franchise_after"].insert_many(arr_fra)